# 📄 结构化版本：生成 AT 指令文档 Notebook

In [79]:
!pip install pandas jinja2 sphinx sphinx-rtd-theme

In [80]:
import os
import json
import re
import pandas as pd
from jinja2 import Environment, Template
from collections import defaultdict

In [81]:
# 写入 conf.py 和 custom.css（如不存在）
conf_path = "docs/source/conf.py"
if not os.path.exists(conf_path):
    os.makedirs(os.path.dirname(conf_path), exist_ok=True)
    with open(conf_path, "w", encoding="utf-8") as f:
        f.write("""
import os
import sys
sys.path.insert(0, os.path.abspath('.'))

project = 'AT Command Manual'
author = 'Your Name'
release = '1.0'

extensions = []
templates_path = ['_templates']
exclude_patterns = []

html_theme = 'sphinx_rtd_theme'
html_static_path = ['_static']
html_css_files = ['custom.css']

def setup(app):
    app.add_css_file('custom.css')
""")
    print("✅ conf.py 写入")
else:
    print("ℹ️ conf.py 已存在")

static_dir = "docs/source/_static"
os.makedirs(static_dir, exist_ok=True)
with open(os.path.join(static_dir, "custom.css"), "w", encoding="utf-8") as f:
    f.write("""
body { font-family: 'Microsoft YaHei', sans-serif; }
.literal-block pre { background-color: #f8f8f8; padding: 0.6em; border: 1px solid #ddd; }
""")
print("✅ custom.css 写入")

ℹ️ conf.py 已存在
✅ custom.css 写入


In [82]:
# 读取 CSV 并展开多类型行逻辑
CSV_PATH = "at_commands_full_template.csv"  # 替换为你的 CSV 文件路径
OUTPUT_DIR = "docs/source"

def expand_multitype_row(row):
    types = [t.strip() for t in str(row.get('命令类型', '')).split(';')]
    fmts = [t.strip() for t in str(row.get('命令格式', '')).split(';')]
    raw_corr = str(row.get('响应校正', '')).strip().strip("'''")
    if raw_corr:
        resps = raw_corr.split(';')
    else:
        resps = [t.strip() for t in str(row.get('响应', '')).split(';')]
    examples = [str(row.get('示例命令', '')).strip()]

    max_len = max(len(types), len(fmts), len(resps), len(examples))
    expanded = []
    for i in range(max_len):
        expanded.append({
            '章节': row.get('章节'),
            '命令': row.get('命令'),
            '命令标题': row.get('命令标题'),
            '命令类型': types[i] if i < len(types) else "",
            '命令格式': fmts[i] if i < len(fmts) else "",
            '响应': resps[i] if i < len(resps) else "",
            '示例命令': examples[i] if i < len(examples) else "",
            '功能描述': row.get('功能描述'),
            '备注': row.get('备注'),
            '参数json': row.get('参数json'),
            '顺序': f"{row.get('顺序')}.{i+1}"
        })
    return expanded

df0 = pd.read_csv(CSV_PATH, dtype=str).fillna("")
expanded = []
for _, r in df0.iterrows():
    expanded.extend(expand_multitype_row(r))
df = pd.DataFrame(expanded)
df.columns = df.columns.str.strip()


In [83]:
# 渲染命令 rst 文件（含子类型和示例命令修正）
TEMPLATE_STRING = """
.. _cmd-{{ command_name | lower }}:

{{ command_name }}：{{ command_title }}
{{ '=' * (command_name | length) }}

{{ description or '' }}

{% for st in subtypes %}
{{ st.type }}命令
{{ (st.type + '命令') | length * '^' }}

**命令：**

::

    {{ st.fmt }}

**响应：**

::

{% for line in st.response.splitlines() %}
    {{ line }}
{% endfor %}

{% endfor %}

参数
^^^^
{% if parameters %}
{% for name, data in parameters.items() %}
- **{{ name }}**：
  {% if data.get('__desc__') %}
    {{ data['__desc__'] }}
  {% endif %}
  {% for key, value in data.items() if key != '__desc__' %}
    - {{ key }}：{{ value }}
  {% endfor %}
{% endfor %}
{% else %}
无
{% endif %}

说明
^^^^
{{ note or '无' }}

示例命令
^^^^^^^^

::

{% for st in subtypes %}
{% if st.example %}
{% for line in st.example.splitlines() %}
    {{ line }}
{% endfor %}
{% endif %}
{% endfor %}
"""

env = Environment()
template = env.from_string(TEMPLATE_STRING)

chapter_commands = defaultdict(list)
chapter_names = []

for chap, grp in df.groupby("章节"):
    chap_name = str(chap).strip()
    chapter_names.append(chap_name)
    chapter_commands.setdefault(chap_name, [])
    chap_dir = os.path.join(OUTPUT_DIR, chap_name)
    os.makedirs(chap_dir, exist_ok=True)

    for _, row in grp.iterrows():
        cmd = str(row["命令"]).strip()
        if cmd not in chapter_commands[chap_name]:
            chapter_commands[chap_name].append(cmd)

        # 不拆分示例命令；保留整块文本
        example_text = str(row.get("示例命令", "") or "").strip()

        st = {
            "type": row.get("命令类型", ""),
            "fmt": row.get("命令格式", ""),
            "response": row.get("响应", ""),
            "example": example_text
        }
        subtypes = [st]

        row0 = grp.iloc[0]
        command_title = str(row0.get("命令标题", "")).strip()
        description = str(row0.get("功能描述", "")).strip()
        note = str(row0.get("备注", "")).strip()
        param_json = row0.get("参数json", "{}")
        try:
            parameters = json.loads(param_json)
        except:
            parameters = {}
        if isinstance(parameters, list):
            pdict = {}
            for p in parameters:
                name = p.get("name", "").strip()
                desc = p.get("desc", "").strip()
                valmap = p.get("valmap", {}) or {}
                pdict[name] = {"__desc__": desc}
                for k, v in valmap.items():
                    pdict[name][k] = v
            parameters = pdict

        content = template.render(
            command_name=cmd,
            command_title=command_title,
            description=description,
            note=note,
            parameters=parameters,
            subtypes=subtypes
        )

        out_path = os.path.join(chap_dir, f"{cmd}.rst")
        with open(out_path, "w", encoding="utf-8") as f:
            f.write(content)

print("✅ RST 文档生成完毕")
print("DEBUG 示例命令:", repr(example_text))

✅ RST 文档生成完毕
DEBUG 示例命令: ''


In [84]:
# 生成章节索引 index.rst
chapter_tmpl = """{{ chapter }}
{{ '=' * (chapter|length) }}

.. toctree::
   :maxdepth: 1

{% for c in cmds %}   {{ c }}
{% endfor %}
"""

main_tmpl = """AT 指令文档
===============

.. toctree::
   :maxdepth: 1
   :caption: 章节目录

{% for ch in chapter_names %}   {{ ch }}/index
{% endfor %}
"""

from jinja2 import Template
for chap, cmds in chapter_commands.items():
    idx = os.path.join(OUTPUT_DIR, chap, 'index.rst')
    with open(idx, 'w', encoding='utf-8') as f:
        f.write(Template(chapter_tmpl).render(chapter=chap, cmds=cmds))

main_idx = os.path.join(OUTPUT_DIR, 'index.rst')
with open(main_idx, 'w', encoding='utf-8') as f:
    f.write(Template(main_tmpl).render(chapter_names=chapter_names))
print("✅ index.rst 完成")

✅ index.rst 完成


In [85]:
!sphinx-build -b html docs/source docs/build/html -c docs/source
print("✅ HTML 构建完成")

Running Sphinx v8.2.3
loading translations [en]... done
loading pickled environment... The configuration has changed (3 options: 'html_css_files', 'html_permalinks_icon', 'jquery_use_sri')
done
building [mo]: targets for 0 po files that are out of date
writing output... 
building [html]: targets for 3 source files that are out of date
updating environment: 0 added, 3 changed, 0 removed
reading sources... [ 33%] 2/AT+CSMS
reading sources... [ 67%] 2/index
reading sources... [100%] index

looking for now-outdated files... none found
pickling environment... done
checking consistency... done
preparing documents... done
copying assets... 
copying static files... 
Writing evaluated template result to C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code\csv2html_2\docs\build\html\_static\basic.css
Writing evaluated template result to C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code\csv2html_2\docs\build\html\_static\documen

C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code\csv2html_2\docs\source\2\AT+CSMS.rst:5: WARNING: Title underline too short.

AT+CSMS：选择短信服务
======= [docutils]
C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code\csv2html_2\docs\source\2\AT+CSMS.rst:11: WARNING: Title underline too short.

测试命令
^^^^ [docutils]
C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code\csv2html_2\docs\source\2\AT+CSMS.rst:68: WARNING: Literal block expected; none found. [docutils]
